In [1]:
from tensorflow.keras.layers import Input,Lambda,Dense,Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt 



ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
IMAGE_SIZE = [224,224]

train_path = "C:/Users/AadithyaVikram/Documents/black fungus/blackfungus/training"

validation_path = "C:/Users/AadithyaVikram/Documents/black fungus/blackfungus/validation"

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255,horizontal_flip=True,zoom_range=0.2)
validation_datagen = ImageDataGenerator(rescale=1./255,horizontal_flip=True,zoom_range=0.2)

training_set = train_datagen.flow_from_directory(
        train_path,target_size=(224,224), batch_size=32,class_mode='categorical',
        subset='training')

validation_set = validation_datagen.flow_from_directory(
        validation_path,target_size=(224,224), batch_size=32,class_mode='categorical',shuffle = True,
        )

Found 41 images belonging to 2 classes.
Found 21 images belonging to 2 classes.


In [4]:
from tensorflow.keras.applications import VGG19
from tensorflow.keras.layers import GlobalAveragePooling2D,Dropout


In [5]:
mv = VGG19(input_shape=IMAGE_SIZE+[3],weights='imagenet',include_top=False)

for layers in mv.layers:
    layers.trainable = False


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [6]:
x = Flatten()(mv.output)
prediction = Dense(2,activation='softmax')(x)


In [7]:
model = Model(inputs=mv.input,outputs=prediction)

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [8]:
import tensorflow as tf

class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self,epoch,logs={}):
        if(logs.get('loss')<=0.05):
            print("\nEnding training")
            self.model.stop_training = True
            callbacks = myCallback()


In [9]:
from tensorflow.keras.optimizers import Adam
model.compile(optimizer=Adam(lr=0.0001),loss='categorical_crossentropy',metrics=['categorical_accuracy'])


In [ ]:
history = model.fit(training_set,
                              validation_data=validation_set,
                              epochs=10,
                              verbose=1,
                              steps_per_epoch=5,
                              validation_steps=5,
                            
                             )



Epoch 1/10
5/5 [==============================] - 39s 8s/step - loss: 0.7037 - categorical_accuracy: 0.5965 - val_loss: 0.6222 - val_categorical_accuracy: 0.6571
Epoch 2/10
5/5 [==============================] - 34s 7s/step - loss: 0.6060 - categorical_accuracy: 0.7582 - val_loss: 0.5273 - val_categorical_accuracy: 0.9048
Epoch 3/10
5/5 [==============================] - 37s 7s/step - loss: 0.5117 - categorical_accuracy: 0.8246 - val_loss: 0.4592 - val_categorical_accuracy: 0.8000
Epoch 4/10
4/5 [=======================>......] - ETA: 3s - loss: 0.4077 - categorical_accuracy: 0.9048

In [ ]:
acc = history.history['categorical_accuracy']
val_acc = history.history['val_categorical_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))


In [ ]:
import matplotlib.pyplot as plt
plt.plot(epochs,acc)
plt.plot(epochs,val_acc)
plt.title("Training and validation Accuracy")
plt.savefig('accuracytrain.png')

plt.plot(epochs,loss)
plt.plot(epochs,val_loss)
plt.title("Training and validation Loss")
plt.savefig('validationaccuracy.png')

model.save("soilnew.h5")